In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/data"

/content/drive/My Drive/Colab Notebooks/data


In [4]:
df =pd.read_csv('men_shoes.csv',low_memory=False)
df.shape

(18280, 48)

In [0]:
def run_model(feads, model = DecisionTreeRegressor(max_depth=5) ):
  X = df[feads].values
  y = df['prices_amountmin'].values
  scores = cross_val_score (model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [9]:
df['brand_cat'] = df.brand.factorize()[0]
feads = ['brand_cat']
run_model(feads)

(-58.38655694633361, 4.223555478221712)

In [11]:
df['brand_cat'] = df.brand.map(lambda x: str(x).lower()).factorize()[0]
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feads, model)

(-57.31783843165656, 4.181246596160967)

In [12]:
df.features.values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       ...,
       '[{"key":"Gender","value":["Unisex"]},{"key":"Frame Style","value":["Wrap"]},{"key":"Polarized Lenses","value":["Polarized lenses"]},{"key":"Color","value":["Multi-color"]}]',


Skonwertujemy features na słownik, aby z niego korzystać dalej

In [0]:
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"', '"'))

In [0]:
df['features_parsed'] = df['features'].map(parse_features)

In [22]:
df['features_parsed'].head()

0    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
1    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
2    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
3    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
4    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
Name: features_parsed, dtype: object

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [26]:
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [27]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [29]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feads: feads[key] if key in feads else np.nan )

In [30]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_condition', 'feat_assembled in country of origin',
       'feat_pronation', 'feat_model no.', 'feat_fabric care instructions',
       'feat_leather :', 'feat_compatible devices', 'feat_fabric type',
       'feat_����������������', 'feat_cm'],
      dtype='object', length=526)

In [31]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [33]:
df.shape[0]

18280

Wartości w kolumenie feat_athlete występują 9 razy
Sprawdźmy które z dodanych kolumn mają dużo wartości (co może sugerować, że do czego się moga przydać)

In [0]:
  keys_stats = {}

  for key in keys:
    keys_stats[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [37]:
{k:v for k,v in keys_stats.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_manufacturer_part_number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

In [49]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
#df[df.brand != df.feat_brand][['brand', 'feat_brand']].head()
df[df.brand != df.feat_brand].shape

(9434, 530)

In [0]:
feads = ['brand_cat', 'feat_color_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
result = run_model(feads, model)

Sprawdzamy za pomocą eli5 które atrybuty wpływają nam na wynik

In [84]:
X = df[feads].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feads)

(-57.13201219072654, 4.160944731064544)


Weight,Feature
0.2584 ± 0.0103,brand_cat
0.1014 ± 0.0114,feat_material_cat
0.0439 ± 0.0021,feat_gender_cat
0.0204 ± 0.0007,feat_brand_cat
0.0071 ± 0.0014,feat_color_cat


In [55]:
feads = ['brand_cat',  'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feads, model)

(-57.20148875865986, 4.149725476371453)

In [56]:
X = df[feads].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feads)

Weight,Feature
0.2595 ± 0.0080,brand_cat
0.1099 ± 0.0134,feat_material_cat
0.0435 ± 0.0055,feat_gender_cat
0.0224 ± 0.0007,feat_brand_cat


Jak widać usunięcie niby małoznaczącego 'color' wpływneło na pogorszenie wyniku. Dla 'manufacturer part number' usunięcie poprawiło

In [58]:
df.brand.value_counts(normalize=True)

nike                           0.097210
puma                           0.033315
ralph lauren                   0.028775
vans                           0.021116
new balance                    0.020295
                                 ...   
silver elements charms         0.000055
pleaser shoes                  0.000055
david spencer                  0.000055
tony sports accessories 888    0.000055
ridge                          0.000055
Name: brand, Length: 1732, dtype: float64

In [60]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [67]:
feads = ['brand_cat',  'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feads, model)

(-57.19331189572881, 4.144629179638678)

In [68]:
X = df[feads].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feads)

Weight,Feature
0.2631 ± 0.0101,brand_cat
0.1087 ± 0.0093,feat_material_cat
0.0436 ± 0.0027,feat_gender_cat
0.0219 ± 0.0012,feat_brand_cat
0.0002 ± 0.0001,feat_sport_cat


In [69]:
feads = ['brand_cat',  'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feads, model)

(-57.16468774262015, 4.235810616060932)

In [70]:
X = df[feads].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feads)

Weight,Feature
0.2600 ± 0.0110,brand_cat
0.1047 ± 0.0084,feat_material_cat
0.0458 ± 0.0047,feat_gender_cat
0.0213 ± 0.0011,feat_brand_cat
0.0069 ± 0.0016,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [76]:
feads_cat = [x for x in df.columns if 'cat' in x]

feads_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_location - country',
 'feat_shoe category',
 'feat_fabrication',
 'feat_multi pack indicator',
 'feat_catalog',
 'feat_location - city/state',
 'feat_certifications and listings',
 'feat_clothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_manufacturer_part_number_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_bracelet style_cat',
 'feat_dial color_cat',
 'feat_product id_cat',
 'feat_gold karat_cat',
 'feat_watch style_cat',
 'feat_protects against_cat',
 'feat_special features_cat',
 'feat_main colour_cat',
 'feat_animal type_cat',
 'feat_recommended location_cat',
 'feat_chest_cat',
 'feat_primary color_cat',
 'feat_material composition_cat',
 'feat_flame resistant_cat',
 'feat_diameter_cat',
 'feat_stock no._cat',
 'feat_heel height_cat',
 'feat_combined shipping_cat',
 'feat_issued/ not-issued_cat',
 'feat_labor_cat',
 'feat_safety features_cat',
 'feat_manuf

In [82]:
feads = ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
#feads += ['feat_brand_cat',
# 'feat_color_cat',
# 'feat_gender_cat',
# 'feat_manufacturer_part_number_cat',
# 'feat_sport_cat',
# 'feat_style_cat',
# 'feat_bracelet style_cat',
# 'feat_dial color_cat',
# 'feat_product id_cat',
# 'feat_gold karat_cat',
# 'feat_watch style_cat',
# 'feat_protects against_cat',
# 'feat_special features_cat',
# 'feat_main colour_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feads, model)

(-57.341289636097635, 4.21571342120217)

In [80]:
X = df[feads].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feads)

Weight,Feature
0.2501 ± 0.0119,brand_cat
0.0970 ± 0.0074,feat_material_cat
0.0113 ± 0.0004,feat_dial color_cat
0.0092 ± 0.0008,feat_gender_cat
0.0069 ± 0.0005,feat_brand_cat
0.0065 ± 0.0015,feat_brand_cat
0.0059 ± 0.0009,feat_color_cat
0.0052 ± 0.0006,feat_gender_cat
0.0047 ± 0.0010,feat_gold karat_cat
0.0038 ± 0.0001,feat_watch style_cat
